CS 4375 - Final Project\
Model 3: EMNIST Balanced (Manual Training)

**Author**: Alec Ibarra\
**Date**: 2024-11-22

In [ ]:
# Install dependencies
%pip install --upgrade pip
%pip install torchvision
%pip install opencv-python-headless
%pip install matplotlib
%pip install pandas
%pip install scikit-learn
%pip install tensorflow[and-cuda]

In [ ]:
# Import dependencies
import os
import matplotlib.pyplot as plt # type: ignore
import numpy as np # type: ignore
import pandas as pd # type: ignore
import tensorflow as tf # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D # type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore

import cv2 # type: ignore
from torchvision import datasets # type: ignore

In [ ]:
# Tensorflow version
print(f"Tensorflow: v{tf.__version__}")

# Check GPU availability
print(f"GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Set seed for reproducibility
SEED = 42
np.random.seed(SEED)

In [ ]:
# Define data transformations
def transform(image):
    image = np.array(image)
    image = cv2.flip(image, 1)
    center = (image.shape[1] // 2, image.shape[0] // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle=90, scale=1.0)
    image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=-1)
    return image

# Load EMNIST Balanced subset
emnist_train = datasets.EMNIST(root='./data', split='balanced', train=True, transform=transform, download=True)
emnist_test = datasets.EMNIST(root='./data', split='balanced', train=False, transform=transform, download=True)

# Convert to numpy arrays for TensorFlow
X_train = np.array([img[0] for img in emnist_train])
y_train = np.array([img[1] for img in emnist_train])
X_test = np.array([img[0] for img in emnist_test])
y_test = np.array([img[1] for img in emnist_test])

# One-hot encode the labels
y_train = to_categorical(y_train, 47)
y_test = to_categorical(y_test, 47)

# Display some info and stats about the dataset
print(f'Training data shape: {X_train.shape}')
print(f'Test data shape: {X_test.shape}')
print(f'Number of classes: {y_train.shape[1]}')

In [ ]:
# Plot some data samples
fig, axes = plt.subplots(1, 15, figsize=(20, 3))
for i, ax in enumerate(axes):
    ax.imshow(X_train[i].squeeze(), cmap='gray')
    ax.set_title(f'Label: {np.argmax(y_train[i])}')
    ax.axis('off')

In [ ]:
# Define specific hyperparameter combinations to test
hyperparameter_combinations = [
    {'conv1':  32, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1':  64, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1':  96, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 128, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 160, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 192, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 224, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 256, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},

    {'conv1': 288, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 320, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 352, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 384, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 416, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 448, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 480, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},


    {'conv1': 512, 'conv2':  32, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2':  64, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2':  96, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 128, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 160, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 192, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 224, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 256, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},

    {'conv1': 512, 'conv2': 288, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 320, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 352, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 384, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 416, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 448, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},
    {'conv1': 512, 'conv2': 480, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0001, 'batch_size': 32},




    {'conv1':  32, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1':  64, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1':  96, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 128, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 160, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 192, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 224, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 256, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},

    {'conv1': 288, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 320, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 352, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 384, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 416, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 448, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 480, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 512, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},


    {'conv1': 512, 'conv2':  32, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2':  64, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2':  96, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 128, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 160, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 192, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 224, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 256, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},

    {'conv1': 512, 'conv2': 288, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 320, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 352, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 384, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 416, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 448, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
    {'conv1': 512, 'conv2': 480, 'dense_units': 512, 'dropout_rate': 0.7, 'learning_rate': 0.0002, 'batch_size': 32},
]

# Define the model
def create_custom_model(conv1, conv2, dense_units, dropout_rate, learning_rate):
    model = Sequential([
        Input(shape=(28, 28, 1)),
        Conv2D(conv1, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(conv2, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(47, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
# Train and evaluate models for each hyperparameter combination
results = []
for i, params in enumerate(hyperparameter_combinations):
    print(f"Training model {i + 1}/{len(hyperparameter_combinations)} with params: {params}")
    model = create_custom_model(
        int(params['conv1']),
        int(params['conv2']),
        int(params['dense_units']),
        float(params['dropout_rate']),
        float(params['learning_rate'])
    )
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=params['batch_size'],
        validation_split=0.2,
        callbacks=[
            EarlyStopping(
                monitor='val_accuracy',
                patience=3,
                restore_best_weights=True,
                verbose=1
            )
        ],
    )
    
    # Evaluate the model
    val_accuracy = max(history.history['val_accuracy'])
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    results.append({
        'val_accuracy': val_accuracy,
        'conv1': params['conv1'],
        'conv2': params['conv2'],
        'dense_units': params['dense_units'],
        'dropout_rate': params['dropout_rate'],
        'learning_rate': params['learning_rate'],
        'epochs': len(history.history['val_accuracy']),
        'batch_size': params['batch_size'],
    })
    print(f"Validation Accuracy: {val_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Save the results to a CSV file
os.makedirs('results/emnist-balanced', exist_ok=True)
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('val_accuracy', ascending=False)
results_df.to_csv(f'results/emnist-balanced/manual-3.csv', index=False)
results_df.head()